<h1>Neural Network - Number of Trips<h1>

In [1]:
from tensorflow import keras
from tensorflow.keras.layers import Dense, Normalization

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import sum,avg,max,min,mean,count
import numpy as np

In [3]:
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "8g")
    .config("spark.sql.parquet.enableVectorizedReader", False)
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/08/16 23:42:51 WARN Utils: Your hostname, Sens-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.11 instead (on interface en0)
22/08/16 23:42:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/16 23:42:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/16 23:42:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
sdf = spark.read.parquet('../data/curated/combined_data')

In [5]:
df = sdf.groupby('Date', 'Hour', 'PU_Location_ID').agg(avg("Temperature_C").alias("Temperature_C"), \
                                                           avg("Humidity_%").alias("Humidity_%"), \
                                                           avg("Speed_kmh").alias("Speed_kmh"), \
                                                           avg("Precip_Rate_mm").alias("Precip_rate_mm"), \
                                                           avg("Driver_pay").alias("Avg_driver_pay"), \
                                                           avg("Day_of_week").alias("Day_of_week"), \
                                                           count('Temperature_C').alias("Num_trips")).toPandas()

In [6]:
df = df.sort_values(['Date', 'PU_Location_ID', 'Hour'], axis = 0)

In [7]:
df = pd.get_dummies(df, columns=['Hour', 'PU_Location_ID', 'Day_of_week'])

In [8]:
TARGET_COLS = ['Num_trips']

train, test = train_test_split(df, train_size=0.8, shuffle = False)

X_train, y_train = train.drop(TARGET_COLS, axis=1).drop(['Avg_driver_pay', 'Date'], axis=1), train[TARGET_COLS]
X_test, y_test = test.drop(TARGET_COLS, axis=1).drop(['Avg_driver_pay', 'Date'], axis=1), test[TARGET_COLS]

In [9]:
norm_layer = Normalization()
norm_layer.adapt(X_train)

Metal device set to: Apple M1


2022-08-16 23:43:50.899139: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-16 23:43:50.899927: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-08-16 23:43:59.168075: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-16 23:43:59.309459: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-16 23:43:59.331493: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [10]:
model = keras.Sequential(
    [   
        norm_layer,
        Dense(10, activation='relu'),
        Dense(1, activation='relu')
    ]
)

In [11]:
model.compile(
    optimizer='adam',
    loss='MSE'
)

In [12]:
history = model.fit(
    x=X_train,
    y=y_train,
    batch_size=16,
    validation_split=0.25,
    epochs=10
)

Epoch 1/10
   15/81606 [..............................] - ETA: 5:04 - loss: 28686.2812  

2022-08-16 23:46:45.002447: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


81605/81606 [============================>.] - ETA: 0s - loss: 3132.8928

2022-08-16 23:51:51.521349: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


81606/81606 [==============================] - 369s 5ms/step - loss: 3132.8906 - val_loss: 2444.6448
Epoch 2/10
81606/81606 [==============================] - 360s 4ms/step - loss: 2828.7024 - val_loss: 2304.0125
Epoch 3/10
81606/81606 [==============================] - 357s 4ms/step - loss: 2602.0500 - val_loss: 2087.0493
Epoch 4/10
81606/81606 [==============================] - 356s 4ms/step - loss: 2863.5769 - val_loss: 2139.2808
Epoch 5/10
81606/81606 [==============================] - 359s 4ms/step - loss: 2550.1360 - val_loss: 1950.5179
Epoch 6/10
81606/81606 [==============================] - 358s 4ms/step - loss: 2450.7212 - val_loss: 1779.1794
Epoch 7/10
81606/81606 [==============================] - 359s 4ms/step - loss: 2030.2279 - val_loss: 1594.7871
Epoch 8/10
81606/81606 [==============================] - 357s 4ms/step - loss: 1748.9528 - val_loss: 1480.2471
Epoch 9/10
81606/81606 [==============================] - 359s 4ms/step - loss: 1578.2190 - val_loss: 1411.0402
Epo

In [13]:
comparison = y_test.iloc[:10].copy()
comparison.loc[:, 'prediction_num_trips'] = model.predict(X_test.head(10))
comparison

1/1 [==============================] - 0s 67ms/step


2022-08-17 00:46:39.127583: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Num_trips,prediction_num_trips
1421589,27,7.321668
1679459,36,38.601170
600889,82,71.196953
600890,105,123.473175
592706,154,182.488129
1420705,182,184.634628
80970,155,153.648834
79654,140,160.079681
337620,160,180.792755
880092,150,188.706757


In [14]:
predictions = model.predict(X_test)
errors = np.array(predictions - y_test)
squared_errors = errors**2
mean_squared_error = squared_errors.mean()

print(f'MSE: {mean_squared_error}')

13601/13601 [==============================] - 21s 2ms/step
MSE: 2199.3950548653816


In [15]:
tot_sum_squares = (np.array(y_test - y_test.mean())**2).sum()
r2 = 1 - (squared_errors.sum() / tot_sum_squares)
print(f'Model R^2: {r2:.4f}')

Model R^2: 0.8155


<h3>Save predictions for further analysis<h3>

In [16]:
import os
if not os.path.exists('../data/curated/model_data'):
    os.makedirs('../data/curated/model_data')

In [17]:
pd.DataFrame(predictions).to_csv('../data/curated/model_data/num_trips_pred_nn.csv')

In [18]:
test.to_csv('../data/curated/model_data/test_data.csv')